In [1]:
import sys
!{sys.executable} -m pip install opencv-python


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: /Users/shy/Projects/clippy/env/bin/python -m pip install --upgrade pip


In [9]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown
import cv2
import shutil
import ffmpeg
import subprocess
import os

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [3]:
import os

GOOGLE_API_KEY= os.environ['GOOGLE_API_KEY']
genai.configure(api_key=GOOGLE_API_KEY)

In [4]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)


models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [5]:
model = genai.GenerativeModel('gemini-pro')

In [9]:
response = model.generate_content("What is the meaning of life?")

In [10]:
to_markdown(response.text)

> The meaning of life is a deeply philosophical question that has been pondered by humans for centuries. There is no single, universally agreed-upon answer, and different people and cultures have come up with varying perspectives on this topic. Some common themes that emerge in discussions about the meaning of life include:
> 
> 1. Purpose and Fulfillment: Many people find meaning in their lives by pursuing goals, passions, or activities that give them a sense of purpose and fulfillment. This could involve contributing to a cause, raising a family, pursuing a career, or engaging in creative endeavors.
> 
> 2. Values and Ethics: Some individuals derive meaning from adhering to certain values and ethical principles. They may believe that their life is meaningful when they live in accordance with their values, such as kindness, integrity, honesty, or compassion.
> 
> 3. Relationships and Connections: For many, the meaning of life is found in the relationships and connections they have with others. They experience love, joy, and a sense of belonging when surrounded by people they care about and who care about them.
> 
> 4. Contribution and Legacy: Some people find meaning in making a contribution to the world or leaving a legacy behind them. They may work to improve their community, advance knowledge, or create something that will benefit future generations.
> 
> 5. Meaningful Experiences: Others believe that the meaning of life lies in the experiences themselves. They seek out activities, moments, or adventures that bring them joy, wonder, awe, or a sense of transcendence.
> 
> It's important to note that the meaning of life is subjective and can vary depending on individual beliefs, values, and experiences. What gives one person a sense of purpose may not resonate with another. The search for meaning is an ongoing journey, and there may be different layers or aspects of meaning that become more or less significant at different stages of a person's life.
> 
> Ultimately, the meaning of life is not something that can be definitively defined or imposed upon others. It is a personal and deeply introspective question that each individual must explore and answer for themselves throughout their lifetime.

In [ ]:
response.prompt_feedback

In [ ]:
response.candidates


[index: 0
content {
  parts {
    text: "**Philosophical Perspectives:**\n\n* **Existentialism:** Life has no inherent meaning, but individuals create their own meaning through choices and actions.\n* **Absurdism:** Life is inherently meaningless, and the search for meaning is futile.\n* **Stoicism:** Life is full of challenges and suffering, but meaning can be found by accepting one\'s fate and living a virtuous life.\n* **Epicureanism:** The meaning of life lies in seeking pleasure and avoiding pain.\n* **Buddhism:** Life is a cycle of suffering caused by desire and attachment. Liberation from suffering (nirvana) is the ultimate goal.\n* **Hinduism:** Life is a journey through multiple reincarnations, with the ultimate goal of reaching moksha (liberation from the cycle of birth and death).\n\n**Scientific Perspectives:**\n\n* **Evolutionary biology:** The purpose of life is to pass on genes through reproduction.\n* **Neuroscience:** Meaning is a subjective experience created by the b

In [ ]:
response = model.generate_content("What is the meaning of life?", stream=True)

## Pro-Vision Video!

In [11]:
model = genai.GenerativeModel('gemini-pro-vision')


In [ ]:
!ls /Users/shy/Desktop/4-19-Sasha-blocked.MP4   

/Users/shy/Desktop/4-19-Sasha-blocked.MP4


In [15]:
video_file_name = "/Users/shy/Desktop/4-19-Sasha-blocked.MP4"

audio_file_name = "/Users/shy/Desktop/4-19-Sasha-blocked.mp3"

subprocess.call(["ffmpeg", "-i", video_file_name, audio_file_name, "-y"])

audio_file = genai.upload_file(path=audio_file_name)

ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.1.0.2.5)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.1.1_4 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopen

In [10]:

# Create or cleanup existing extracted image frames directory.
FRAME_EXTRACTION_DIRECTORY = "frames"
FRAME_PREFIX = "_frame"
def create_frame_output_dir(output_dir):
  if not os.path.exists(output_dir):
    os.makedirs(output_dir)
  else:
    shutil.rmtree(output_dir)
    os.makedirs(output_dir)

def extract_frame_from_video(video_file_path):
  print(f"Extracting {video_file_path} at 1 frame per second. This might take a bit...")
  create_frame_output_dir(FRAME_EXTRACTION_DIRECTORY)
  vidcap = cv2.VideoCapture(video_file_path)
  fps = vidcap.get(cv2.CAP_PROP_FPS)
  frame_duration = 1 / fps  # Time interval between frames (in seconds)
  output_file_prefix = os.path.basename(video_file_path).replace('.', '_')
  frame_count = 0
  count = 0
  while vidcap.isOpened():
      success, frame = vidcap.read()
      if not success: # End of video
          break
      if int(count / fps) == frame_count: # Extract a frame every second
          min = frame_count // 60
          sec = frame_count % 60
          time_string = f"{min:02d}:{sec:02d}"
          image_name = f"{output_file_prefix}{FRAME_PREFIX}{time_string}.jpg"
          output_filename = os.path.join(FRAME_EXTRACTION_DIRECTORY, image_name)
          cv2.imwrite(output_filename, frame)
          frame_count += 1
      count += 1
  vidcap.release() # Release the capture object\n",
  print(f"Completed video frame extraction!\n\nExtracted: {frame_count} frames")

extract_frame_from_video(video_file_name)

Extracting /Users/shy/Desktop/4-19-Sasha-blocked.MP4 at 1 frame per second. This might take a bit...
Completed video frame extraction!

Extracted: 8 frames


In [11]:
class File:
  def __init__(self, file_path: str, display_name: str = None):
    self.file_path = file_path
    if display_name:
      self.display_name = display_name
    self.timestamp = get_timestamp(file_path)

  def set_file_response(self, response):
    self.response = response

def get_timestamp(filename):
  """Extracts the frame count (as an integer) from a filename with the format
     'output_file_prefix_frame00:00.jpg'.
  """
  parts = filename.split(FRAME_PREFIX)
  if len(parts) != 2:
      return None  # Indicates the filename might be incorrectly formatted
  return parts[1].split('.')[0]

# Process each frame in the output directory
files = os.listdir(FRAME_EXTRACTION_DIRECTORY)
files = sorted(files)
files_to_upload = []
for file in files:
  files_to_upload.append(
      File(file_path=os.path.join(FRAME_EXTRACTION_DIRECTORY, file)))

# Upload the files to the API
uploaded_files = []
print(f'Uploading {len(files_to_upload)} files. This might take a bit...')

for file in files_to_upload:
  print(f'Uploading: {file.file_path}...')
  response = genai.upload_file(path=file.file_path)
  file.set_file_response(response)
  uploaded_files.append(file)

print(f"Completed file uploads!\n\nUploaded: {len(uploaded_files)} files")
     

Uploading 8 files. This might take a bit...
Uploading: frames/4-19-Sasha-blocked_MP4_frame00:00.jpg...
Uploading: frames/4-19-Sasha-blocked_MP4_frame00:01.jpg...
Uploading: frames/4-19-Sasha-blocked_MP4_frame00:02.jpg...
Uploading: frames/4-19-Sasha-blocked_MP4_frame00:03.jpg...
Uploading: frames/4-19-Sasha-blocked_MP4_frame00:04.jpg...
Uploading: frames/4-19-Sasha-blocked_MP4_frame00:05.jpg...
Uploading: frames/4-19-Sasha-blocked_MP4_frame00:06.jpg...
Uploading: frames/4-19-Sasha-blocked_MP4_frame00:07.jpg...
Completed file uploads!

Uploaded: 8 files


In [12]:
# List files uploaded in the API
for n, f in zip(range(len(uploaded_files)), genai.list_files()):
  print(f.uri)

https://generativelanguage.googleapis.com/v1beta/files/jlsp82dqvm0e
https://generativelanguage.googleapis.com/v1beta/files/rrz4cbk64vnb
https://generativelanguage.googleapis.com/v1beta/files/ljpl8398vjro
https://generativelanguage.googleapis.com/v1beta/files/uvwqs6o38j75
https://generativelanguage.googleapis.com/v1beta/files/bgyf1nf3w6o8
https://generativelanguage.googleapis.com/v1beta/files/6s8ferghwpnf
https://generativelanguage.googleapis.com/v1beta/files/7grp1u6hwvz7
https://generativelanguage.googleapis.com/v1beta/files/rc643fgkbjo0


In [16]:
# Create the prompt.
prompt = "Describe this video."

# Set the model to Gemini 1.5 Pro.
model = genai.GenerativeModel(model_name="models/gemini-1.5-pro-latest")

# Make GenerateContent request with the structure described above.
def make_request(prompt, files):
  request = [prompt]
  for file in files:
    request.append(file.timestamp)
    request.append(file.response)
  return request

# Make the LLM request.
request = make_request(prompt, uploaded_files) + [audio_file]
response = model.generate_content(request,
                                  request_options={"timeout": 600})
print(response.text)

A person is talking about using the "magic of Gemini" to create a highlight reel.


In [ ]:
print(f'Deleting {len(uploaded_files)} images. This might take a bit...')
for file in uploaded_files:
  genai.delete_file(file.response.name)
  print(f'Deleted {file.file_path} at URI {file.response.uri}')
print(f"Completed deleting files!\n\nDeleted: {len(uploaded_files)} files")

Deleting 8 images. This might take a bit...
Deleted frames/4-19-Sasha-blocked_MP4_frame00:00.jpg at URI https://generativelanguage.googleapis.com/v1beta/files/8z7zppxtdqox
Deleted frames/4-19-Sasha-blocked_MP4_frame00:01.jpg at URI https://generativelanguage.googleapis.com/v1beta/files/mx54h3r1s3n2
Deleted frames/4-19-Sasha-blocked_MP4_frame00:02.jpg at URI https://generativelanguage.googleapis.com/v1beta/files/olakdz0bwtwn
Deleted frames/4-19-Sasha-blocked_MP4_frame00:03.jpg at URI https://generativelanguage.googleapis.com/v1beta/files/yz7sr6msgvye
Deleted frames/4-19-Sasha-blocked_MP4_frame00:04.jpg at URI https://generativelanguage.googleapis.com/v1beta/files/fnvdut3okxeq
Deleted frames/4-19-Sasha-blocked_MP4_frame00:05.jpg at URI https://generativelanguage.googleapis.com/v1beta/files/cu85rdj4fws2
Deleted frames/4-19-Sasha-blocked_MP4_frame00:06.jpg at URI https://generativelanguage.googleapis.com/v1beta/files/8dphby54g4gp
Deleted frames/4-19-Sasha-blocked_MP4_frame00:07.jpg at URI 